<a href="https://colab.research.google.com/github/Archandra12/Deeplearning-Project/blob/codes/Embedding_Concept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Practice tasks to understand word embeddings ad their use in RNN
- Model can learn embeddings during Model training, or
- Model can use a pretrained embeddings to train the model

We are taking movie ratings data which has 50000 reviews. They are 50% positive and 50% negative reviews. We are breaking data into two parts for training and Testing. Each set has equal proportion of positive or negative reivews.

####Loading libraries

In [2]:
#Pandas libraries
import pandas as pds
import numpy as np

#keras libraries
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [3]:
#method to load file
def load_data(file):
  sheet1 = pds.read_excel(file, sheet_name = 0)  
  sheet2 = pds.read_excel(file, sheet_name = 1)  
  data = pds.concat([sheet1, sheet2], axis=0)
  return data
#Loading Training Data
newData = load_data('/content/drive/MyDrive/LSTM-Dataset/aclImdb/train/mergetest.xlsx')
newData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 0 to 12499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   25000 non-null  object
 1   Review  25000 non-null  object
dtypes: object(2)
memory usage: 585.9+ KB


####Separating reviews and labels

In [4]:
docs = newData.iloc[:, newData.columns!='Label']
labels = newData.iloc[:, newData.columns!='Label']

In [5]:
docs.head(5)

,Review
0,Story of a man who has unnatural feelings for ...
1,Airport '77 starts as a brand new luxury 747 p...
2,This film lacked something I couldn't put my f...
3,"Sorry everyone,,, I know this is supposed to b..."
4,When I was little my parents took me along to ...


####Cleaning and tokenizing the review

In [8]:
from string import punctuation
from collections import Counter
import string
def tokenize_reviews(docs):
  review_text = docs.iloc[:, docs.columns=='Review'].replace('--', ' ')
  review_text = review_text['Review'].str.lower()
  review_text = [c for c in review_text if c not in punctuation]
  all_text2 = ' '.join(review_text)
  #print(all_text2)
  # create a list of words
  words = all_text2.split()
  table = str.maketrans('', '', string.punctuation)
  words = [w.translate(table) for w in words]
  words = [word for word in words if word.isalpha()]
  # Count all the words using Counter Method
  count_words = Counter(words)
  total_words = len(words)
  #we are taking top 10000 words based on their frequency of occurances
  sorted_words = count_words.most_common(10000)
  vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}
  len(vocab_to_int)
  encoded_docs = []
  for review in docs['Review']:
    r = []
    for w in review.split():
      if w in vocab_to_int:
        r.append(vocab_to_int[w])
      else:
        r.append(0)      
    encoded_docs.append(r)
  return encoded_docs
encoded_docs = tokenize_reviews(docs)
print (len(encoded_docs[0]))
print (encoded_docs[1])
print (encoded_docs[2])

112
[0, 0, 494, 14, 3, 3414, 154, 8383, 0, 1641, 6, 4765, 57, 16, 4355, 5719, 0, 135, 0, 5, 993, 4858, 0, 0, 0, 0, 36, 6, 1495, 97, 0, 3, 730, 4, 0, 5, 24, 3477, 7, 0, 4, 8, 106, 2998, 5, 1, 1038, 14, 3, 0, 78, 20, 2042, 6, 0, 573, 0, 0, 0, 0, 39, 0, 0, 8383, 0, 290, 126, 14, 4217, 18, 0, 1, 1641, 6, 0, 32, 1, 0, 0, 0, 0, 0, 24, 104, 0, 0, 0, 0, 0, 0, 0, 0, 36, 3666, 1, 5977, 0, 1020, 45, 16, 2661, 0, 34, 1272, 5, 1997, 1, 4355, 0, 0, 1496, 20, 3, 0, 1641, 3459, 20, 33, 4255, 1079, 18, 130, 244, 24, 4629, 0, 209, 1823, 33, 3199, 0, 7, 1, 0, 0, 1878, 1143, 4, 1, 1641, 5541, 8, 6480, 77, 1, 2023, 112, 8, 7837, 5, 1, 1319, 204, 4630, 7, 1, 743, 4, 1, 0, 0, 0, 986, 7, 345, 0, 1091, 0, 7, 0, 251, 0, 125, 0, 1927, 126, 258, 1, 686, 0, 15, 1, 0, 14, 34, 0, 325, 16, 61, 820, 604, 0, 0, 0, 611, 479, 1, 1037, 265, 0, 0, 0, 10, 323, 759, 5, 1, 0, 1704, 765, 0, 0, 13, 514, 32, 0, 0, 0, 130, 271, 172, 38, 0, 8092, 0, 0, 129, 0, 0, 6, 98, 422, 4, 1554, 347, 8, 6, 438, 254, 21, 2580, 15, 1, 204, 0, 0

###Encoding Labels

In [9]:
encoded_labels = [1 if label =='positive' else 0 for label in newData['Label']]
encoded_labels = np.array(encoded_labels)
encoded_labels

array([0, 0, 0, ..., 0, 0, 0])

###Preparing Input for Model
Padding the encoded reviews to ensure all reviews are of same size in terms of number of tokens. We are keeping total number of tokens in each review as 500

In [101]:
max_length = 500
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[  74    4    2 ...    0    0    0]
 [6441   16  166 ... 6441    0    0]
 [   9   23 3805 ...    0    0    0]
 ...
 [   8  225 2769 ...    0    0    0]
 [   2 1152  372 ...    0    0    0]
 [   0  713  658 ...    0    0    0]]


In [102]:
padded_docs.shape

(25000, 500)

###Define Model - Embedding layer learns along the model

In [106]:
# define the model - model is mainly trying to learn embedding nothing much
vocab_size=10001 #needs to be 1 more than number of words in the vocab
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 500, 32)           320032    
_________________________________________________________________
flatten_12 (Flatten)         (None, 16000)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 16001     
Total params: 336,033
Trainable params: 336,033
Non-trainable params: 0
_________________________________________________________________
None


###Training the Model

In [107]:
# fit the model
model.fit(padded_docs, encoded_labels, epochs=50, verbose=0, validation_split=0.2)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, encoded_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


###Preparing a Model using pretrained Embeddings
 **Took glove.6B.100d.txt for this from GloVe Embeddings

In [108]:
from numpy import array
from numpy import asarray
from numpy import zeros
embeddings_index = dict()
f = open('/content/drive/MyDrive/LSTM-Dataset/aclImdb/train/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((10001, 100))
for word, i in vocab_to_int.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector


Loaded 400000 word vectors.


In [109]:
embedding_matrix.shape

(10001, 100)

###Define Model with pre-trained Embeddings

In [111]:
# define model
vocab_size=10001
model = Sequential()
#vocab size = number of words in entire training dataset, 100 is number of weights for each word, input length = number of tokens in padded_reviews
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=500, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 500, 100)          1000100   
_________________________________________________________________
flatten_14 (Flatten)         (None, 50000)             0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 50001     
Total params: 1,050,101
Trainable params: 50,001
Non-trainable params: 1,000,100
_________________________________________________________________
None


###Training the model with pre-trained embeddings

In [112]:
# fit the model
model.fit(padded_docs, encoded_labels, epochs=50, verbose=0, validation_split=0.2)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, encoded_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


###Checking Model performance on Test data

In [113]:
#Loading Training Data
testData = load_data('/content/drive/MyDrive/LSTM-Dataset/aclImdb/train/MovieRating_Test.xlsx')
testData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24999 entries, 0 to 12499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   24999 non-null  object
 1   Review  24999 non-null  object
dtypes: object(2)
memory usage: 585.9+ KB


In [114]:
encoded_test_docs = tokenize_reviews(testData)
print (len(encoded_test_docs))
print (encoded_test_docs[1])
print (encoded_test_docs[2])

24999
[9, 6, 31, 701, 4, 140, 1, 2305, 4, 251, 126, 20, 1, 3318, 4434, 3, 1564, 214, 60, 166, 248, 150, 706, 2, 528, 425, 4, 1, 94, 0, 2518, 4, 0, 3, 2555, 0, 9613, 229, 0, 98, 285, 129, 11, 32, 20, 2102, 4, 571, 3, 130, 472, 81, 1272, 15, 9, 682, 142, 64, 169, 713, 3212, 0, 39, 107, 169, 1202, 5788, 16, 0, 39, 0, 5938, 1, 0, 2282, 3085, 39, 3067, 16, 2555, 0, 3, 64, 1, 151, 7373, 0, 2233, 0, 500, 891, 148, 9, 24, 0, 28, 1, 0, 3, 146, 127, 1587, 47, 1, 2698, 1119, 0, 13, 382, 7, 9, 2865, 3, 140, 1, 2698, 119, 27, 204, 293, 1, 2540, 161, 0, 33, 2412, 0, 154, 24, 189, 120, 15, 1119, 0, 40, 101, 365, 1, 2540, 161, 2413, 846, 3, 28, 225, 7, 2412, 23, 122, 7138, 65, 97, 10, 612, 0, 12, 2239, 9, 6, 0, 251, 4323, 50, 20, 3980, 132, 126, 5, 2036, 3, 42, 22, 60, 170, 5, 64, 9, 682, 107, 0, 2103, 65, 6, 2657, 2, 0, 950, 17, 40, 132, 130, 3, 2, 132, 1705, 1, 59, 167, 11, 97, 9, 28, 35, 248, 150, 13, 2, 520, 889, 18, 1, 397, 70, 1, 801, 13, 200, 0, 65, 255, 586, 5, 252, 61, 16, 1, 563, 24, 474, 70

In [117]:
encoded_test_labels = [1 if label =='positive' else 0 for label in testData['Label']]
encoded_test_labels = np.array(encoded_test_labels)
encoded_test_labels

array([0, 0, 0, ..., 1, 1, 1])

In [118]:
max_length = 500
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')
print(padded_test_docs)

[[ 300  307  505 ...    0    0    0]
 [   9    6   31 ...    0    0    0]
 [  87    4   35 ...    0    0    0]
 ...
 [  33   50 6004 ...   34  291  741]
 [   9  149   15 ...    0    0    0]
 [ 189  120    9 ...    0    0    0]]


###Predicting on test data

In [121]:
#ynew = model.predict_classes(padded_test_docs)
ynew = np.argmax(model.predict(padded_test_docs), axis=-1)
# show the inputs and predicted outputs
#for i in range(len(padded_test_docs)):
#	print("X=%s, Labels=%s, Predicted=%s" % (padded_test_docs[i], encoded_test_labels[i], ynew[i]))

In [128]:
testData['Predicted_Label'] = ['negative' if i==0 else 'positive' for i in ynew]
testData[testData['Label'] != testData['Predicted_Label']].shape, testData.shape

((12500, 3), (24999, 3))

##How to save and reuse models in Keras

Model weights are saved to HDF5 format. This is a grid format that is ideal for storing multi-dimensional arrays of numbers

The model structure can be described and saved using two different formats: JSON and YAML.

In order to do that we need to install __h5py__

In [129]:
!sudo pip install h5py

Saving Model in JASON and weights in HDF5 format (this helps in storing weights in multidimensions

In [130]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("Embeddings_Conceptmodel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Embeddings_Conceptmodel.h5")
print("Saved model to disk")


Saved model to disk


If we want to create new model using saved JASON and weights We can do that as below

In [132]:
# load json and create model
json_file = open('/content/Embeddings_Conceptmodel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/Embeddings_Conceptmodel.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
loaded_model.fit(padded_docs, encoded_labels, epochs=50, verbose=0, validation_split=0.2)
score = loaded_model.evaluate(padded_docs, encoded_labels, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 100.00%


###Saving the Model in YAML

In [133]:
!pip install PyYAML

In [134]:
from keras.models import model_from_yaml
# serialize model to YAML
model_yaml = model.to_yaml()
with open("Embeddings_Conceptmodel.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("Embeddings_Conceptmodel.h5")
print("Saved model to disk")


Saved model to disk


###To load the model from yaml saved on local path

In [137]:
# load yaml and create model
yaml_file = open('/content/Embeddings_Conceptmodel.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("/content/Embeddings_Conceptmodel.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
loaded_model.fit(padded_docs, encoded_labels, epochs=50, verbose=0, validation_split=0.2)
score = loaded_model.evaluate(padded_docs, encoded_labels, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 100.00%


###Save Model Weights and Architecture Together

Keras also supports a simpler interface to save both the model weights and model architecture together into a single H5 file.

Saving the model in this way includes everything we need to know about the model, including:

 - Model weights.
 - Model architecture.
 - Model compilation details (loss and metrics).
 - Model optimizer state.
 
This means that we can load and use the model directly, without having to re-compile it as we did above.

In [138]:
model.save("Embedding_Conceptmodel_Save.h5")

Need to perform following steps when we need to load model and use it

In [140]:
from keras.models import load_model 
# load model
loaded_model = load_model('Embedding_Conceptmodel_Save.h5')
# summarize model.
loaded_model.summary()
loaded_model.fit(padded_docs, encoded_labels, epochs=50, verbose=0, validation_split=0.2)
score = loaded_model.evaluate(padded_docs, encoded_labels, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 500, 100)          1000100   
_________________________________________________________________
flatten_14 (Flatten)         (None, 50000)             0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 50001     
Total params: 1,050,101
Trainable params: 50,001
Non-trainable params: 1,000,100
_________________________________________________________________
accuracy: 100.00%


##Text Analysis - Prediction on Republic

A language model can predict the probability of the next word in the sequence, based on the words already observed in the sequence.

Neural network models are a preferred method for developing statistical language models because they can use a distributed representation where different words with similar meanings have similar representation and because they can use a large context of recently observed words when making predictions.


We will perform following:
- Data Preparation
- Train Language Model
- Use Language Model

Here is a direct link to the clean version of the data file:

###Data Preparation
We will start by preparing the data for modeling.

We could see following from a quick look on the data:

 - Book/Chapter headings (e.g. “BOOK I.”).
 - British English spelling (e.g. “honoured”)
 - Lots of punctuation (e.g. “–“, “;–“, “?–“, and more)
 - Strange names (e.g. “Polemarchus”).
 - Some long monologues that go on for hundreds of lines.
 - Some quoted dialog (e.g. ‘…’)
 - These observations, and more, suggest at ways that we may wish to prepare the text data.


###Language Model Design
We will develop a model of the text that we can then use to generate new sequences of text.

The language model will be statistical and will predict the probability of each word given an input sequence of text. The predicted word will be fed in as input to in turn generate the next word.

__A key design decision is how long the input sequences should be. They need to be long enough to allow the model to learn the context for the words to predict. This input length will also define the length of seed text used to generate new sequences when we use the model.__

There is no correct answer. With enough time and resources, we could explore the ability of the model to learn with differently sized input sequences. We will pick a length of 50 words for the length of the input sequences, somewhat arbitrarily.

To keep the example brief, we will let all of the text flow together and train the model to predict the next word across sentences, paragraphs, and even books or chapters in the text.

Now that we have a model design, we can look at transforming the raw text into sequences of 50 input words to 1 output word, ready to fit a model.

Load Text
The first step is to load the text into memory.

We can develop a small function to load the entire text file into memory and return it. The function is called load_doc() and is listed below. Given a filename, it returns a sequence of loaded text.

In [2]:
#/content/drive/MyDrive/LSTM-Dataset/republic_clean.txt
def load_doc(filename):
  # open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [3]:
# load document
in_filename = '/content/drive/MyDrive/LSTM-Dataset/republic_clean.txt'
doc = load_doc(in_filename)
print(doc[:200])

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what


###Clean Text
We need to transform the raw text into a sequence of tokens or words that we can use as a source to train the model.

Based on reviewing the raw text (above), below are some specific operations we will perform to clean the text. 

 - Replace ‘–‘ with a white space so we can split words better.
 - Split words based on white space.
 - Remove all punctuation from words to reduce the vocabulary size (e.g. ‘What?’ becomes ‘What’).
 - Remove all words that are not alphabetic to remove standalone punctuation tokens.
 - Normalize all words to lowercase to reduce the vocabulary size.
 - Vocabulary size is a big deal with language modeling. A smaller vocabulary results in a smaller model that trains faster.

We can implement each of these cleaning operations in this order in a function. Below is the function clean_doc() that takes a loaded document as an argument and returns an array of clean tokens.

In [4]:
import string

# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid',


Next, we can look at shaping the tokens into sequences and saving them to file.

Save Clean Text
We can organize the long list of tokens into sequences of 50 input words and 1 output word.

That is, sequences of 51 words.

We can do this by iterating over the list of tokens from token 51 onwards and taking the prior 50 tokens as a sequence, then repeating this process to the end of the list of tokens.

We will transform the tokens into space-separated strings for later storage in a file.


In [5]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 118634


####Saving the Data after cleaning

In [6]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
# save sequences to file
out_filename = '/content/drive/MyDrive/LSTM-Dataset/republic_sequences.txt'
save_doc(sequences, out_filename)

###Train Language Model
We can now train a statistical language model from the prepared data.

The model we will train is a neural language model. It has a few unique characteristics:

 - It uses a distributed representation for words so that different words with similar meanings will have a similar representation.
 - It learns the representation at the same time as learning the model.
 - It learns to predict the probability for the next word using the context of the last 100 words.
 
Specifically, we will use an Embedding Layer to learn the representation of words, and a Long Short-Term Memory (LSTM) recurrent neural network to learn to predict words based on their context.

Let’s start by loading our training data.

Load Sequences
We can load our training data using the load_doc() function we developed in the previous section.

Once loaded, we can split the data into separate training sequences by splitting based on new lines.

In [7]:
# load
in_filename = '/content/drive/MyDrive/LSTM-Dataset/republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
lines[:3]

['book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was',
 'i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted',
 'i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with']

####Encode Sequences
The word embedding layer expects input sequences to be comprised of integers.

We can map each word in our vocabulary to a unique integer and encode our input sequences. Later, when we make predictions, we can convert the prediction to numbers and look up their associated words in the same mapping.

To do this encoding, we will use the __Tokenizer__ class in the Keras API.

First, the Tokenizer must be trained on the entire training dataset, which means it finds all of the unique words in the data and assigns each a unique integer.

We can then use the fit Tokenizer to encode all of the training sequences, converting each sequence from a list of words to a list of integers.

In [8]:
from keras.preprocessing.text import Tokenizer
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [9]:
#to get the mapping of words to integer
tokenizer.word_index
#to know the length of vocabulary as we would need in embedding layer
vocab_size = len(tokenizer.word_index) + 1
vocab_size

7412

####Sequence Inputs and Output
Now that we have encoded the input sequences, we need to separate them into input (X) and output (y) elements.

We can do this with array slicing.

After separating, we need to one hot encode the output word. This means converting it from an integer to a vector of 0 values, one for each word in the vocabulary, with a 1 to indicate the specific word at the index of the words integer value.

This is so that the model learns to predict the probability distribution for the next word and the ground truth from which to learn from is 0 for all words except the actual word that comes next.

Keras provides the to_categorical() that can be used to one hot encode the output words for each input-output sequence pair.

Finally, we need to specify to the Embedding layer how long input sequences are. We know that there are 50 words because we designed the model, but a good generic way to specify that is to use the second dimension (number of columns) of the input data’s shape. That way, if you change the length of sequences when preparing data, you do not need to change this data loading code; it is generic.

In [10]:
# separate into input and output
from numpy import array
from numpy import asarray
from numpy import zeros

from keras.utils.np_utils import to_categorical
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
print(X.shape, y.shape)

(118634, 50) (118634, 7412)


###Fit Model
We can now define and fit our language model on the training data.

The learned embedding needs to know the size of the vocabulary and the length of input sequences as previously discussed. It also has a parameter to specify how many dimensions will be used to represent each word. That is, the size of the embedding vector space.

Common values are 50, 100, and 300. We will use 50 here, but consider testing smaller or larger values.

We will use a two LSTM hidden layers with 100 memory cells each. More memory cells and a deeper network may achieve better results.

A dense fully connected layer with 100 neurons connects to the LSTM hidden layers to interpret the features extracted from the sequence. The output layer predicts the next word as a single vector the size of the vocabulary with a probability for each word in the vocabulary. A softmax activation function is used to ensure the outputs have the characteristics of normalized probabilities.

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            370600    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 7412)              748612    
Total params: 1,270,112
Trainable params: 1,270,112
Non-trainable params: 0
_________________________________________________________________
None


Training the model

In [12]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=10)

Epoch 1/10
927/927 [==============================] - 269s 267ms/step - loss: 6.4775 - accuracy: 0.0604
Epoch 2/10
927/927 [==============================] - 245s 265ms/step - loss: 5.7303 - accuracy: 0.1001
Epoch 3/10
927/927 [==============================] - 245s 264ms/step - loss: 5.4750 - accuracy: 0.1275
Epoch 4/10
927/927 [==============================] - 243s 263ms/step - loss: 5.2907 - accuracy: 0.1449
Epoch 5/10
927/927 [==============================] - 243s 262ms/step - loss: 5.1603 - accuracy: 0.1528
Epoch 6/10
927/927 [==============================] - 246s 266ms/step - loss: 5.0568 - accuracy: 0.1617
Epoch 7/10
927/927 [==============================] - 244s 263ms/step - loss: 4.9781 - accuracy: 0.1666
Epoch 8/10
927/927 [==============================] - 246s 265ms/step - loss: 4.8890 - accuracy: 0.1717
Epoch 9/10
927/927 [==============================] - 244s 263ms/step - loss: 4.8159 - accuracy: 0.1755
Epoch 10/10
927/927 [==============================] - 244s 263m

###Save Model
At the end of the run, the trained model is saved to file.

Here, we use the Keras model API to save the model to the file ‘model.h5‘ in the current working directory.

Later, when we load the model to make predictions, we will also need the mapping of words to integers. This is in the Tokenizer object, and we can save that too using Pickle.

In [19]:
# save the model to file
from pickle import dump
model.save('/content/drive/MyDrive/LSTM-Dataset/languagemodel.h5')
# save the tokenizer
dump(tokenizer, open('/content/drive/MyDrive/LSTM-Dataset/tokenizer.pkl', 'wb'))

###Use Language Model
Now that we have a trained language model, we can use it.

In this case, we can use it to generate new sequences of text that have the same statistical properties as the source text.

This is not practical, at least not for this example, but it gives a concrete example of what the language model has learned.

We will start by loading the training sequences again.

In [17]:
# load cleaned text sequences
in_filename = '/content/drive/MyDrive/LSTM-Dataset/republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

We need the text so that we can choose a source sequence as input to the model for generating a new sequence of text.

The model will require 50 words as input.

Later, we will need to specify the expected length of input. We can determine this from the input sequences by calculating the length of one line of the loaded data and subtracting 1 for the expected output word that is also on the same line.

In [56]:
seq_length = len(lines[0].split()) - 1
seq_length

50

###Load Model
We can now load the model from file.

Keras provides the load_model() function for loading the model, ready for use.

In [24]:
from random import randint
from pickle import load
from keras.models import load_model
loaded_model = load_model('/content/drive/MyDrive/LSTM-Dataset/languagemodel.h5')

 Load the tokenizer from file using the Pickle API

In [25]:
# load the tokenizer
tokenizer = load(open('/content/drive/MyDrive/LSTM-Dataset/tokenizer.pkl', 'rb'))

###Generate Text
The first step in generating text is preparing a seed input.

We will select a random line of text from the input text for this purpose. Once selected, we will print it so that we have some idea of what was used.

In [36]:
print(lines[0])
print(lines[118633])

book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was
one another and to the gods both while remaining here and when like conquerors in the games who go round to gather gifts we receive our reward and it s hall be well with us both in this life and in the pilgrimage of a thousand years which we have been


In [53]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
print(len(seed_text), len(lines))
print(randint(0,len(lines)))

with barbarians as the hellenes now deal with one another then let us enact this law also for our guardians that they are neither to devastate the lands of hellenes nor to burn their houses agreed and we may agree also in thinking that these like all our previous enactments are

278 118634
29720


Next, we can generate new words, one at a time.

First, the seed text must be encoded to integers using the same tokenizer that we used when training the model.

In [74]:
encoded = tokenizer.texts_to_sequences([seed_text])[0]

In [75]:
from keras.preprocessing.sequence import pad_sequences
encoded = pad_sequences([encoded], maxlen=seq_length)


The model can predict the next word directly by calling model.predict_classes() that will return the index of the word with the highest probability.

In [76]:
# predict probabilities for each word
import numpy as np 
#yhat = loaded_model.predict_classes(encoded, verbose=0)
yhat = np.argmax(model.predict(encoded), axis=-1)

We can then look up the index in the Tokenizers mapping to get the associated word.

In [77]:
out_word = ''
for word, index in tokenizer.word_index.items():
	if index == yhat:
		out_word = word
		break
out_word

'the'

Putting it all together as below

In [119]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
  result = list()
  in_text = seed_text
  #print(in_text)
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    
    yhat = np.argmax(model.predict(encoded), axis=-1)    
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        #print(yhat, out_word)
        break
    in_text += ' ' + out_word
    #print(in_text)
    result.append(out_word)
  return ' '.join(result)  
# generate new text
print('Original text:')
print(seed_text)
generated = generate_seq(loaded_model, tokenizer, seq_length, seed_text, 50)
print('Generated text:')
print(generated)

Original text:
rulers in a state those to whom you would entrust the office of determining suits at law certainly and are suits decided on any other ground but that a man may neither take what is anothers nor be deprived of what is his own yes that is their principle which is
Generated text:
the same and the same and the same and the same and the same of the state and the same of the state and the same of the state and the same of the state and the same of the state and the same of the state and the same
